<a href="https://colab.research.google.com/github/CaroBrambila/M3-Actividad-3/blob/main/Regresio%CC%81n_logistica_mexico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
#Cargamos librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [133]:
#Carga desde un archivo .csv sin indice y eliminar filas innecesarias
mexico=pd.read_csv("mexico_limpio.csv")
mexico.head(5)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,host_response_rate,last_review,reviews_per_month,calculated_host_listings_count,availability_365,host_acceptance_rate,accommodates
0,35797,Villa Dante,153786,Dici,Cuajimalpa de Morelos,19.38283,-99.27178,Entire home/apt,3663.0,1,0,2022-01-01 00:00:00,1.70,1,363,0.0,4
1,898633,City SkyLine View from 1 Bedroom Apt in CDMX,4796027,Marco,Cuauhtémoc,19.42063,-99.16586,Entire home/apt,1652.0,3,56,2024-04-17,0.41,3,262,3.0,4
2,913084,15 blocks from Paseo de la Reforma,4647293,Martina,Miguel Hidalgo,19.43956,-99.17263,Entire home/apt,814.0,2,161,2024-05-21,1.19,1,236,6.0,2
3,67703,"2 bedroom apt. deco bldg, Condesa",334451,Nicholas,Cuauhtémoc,19.41152,-99.16857,Entire home/apt,1923.0,2,49,2024-05-05,0.32,2,361,4.0,4
4,939721,Radiant Apartment with Terrace in Roma Norte,4821807,Alex,Cuauhtémoc,19.42288,-99.15488,Entire home/apt,1636.0,3,526,2024-06-04,3.85,3,276,22.0,4


In [134]:
#Rellenamos valores nulos
mexico= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')

<ipython-input-134-cee78b2924db>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexico= mexico.fillna(method='bfill')
<ipython-input-134-cee78b2924db>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexicof= mexico.fillna(method='ffill')


In [135]:
mexico['price'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 26015 entries, 0 to 26014
Series name: price
Non-Null Count  Dtype  
--------------  -----  
26015 non-null  float64
dtypes: float64(1)
memory usage: 203.4 KB


**1 categorica**

In [136]:
# Crear una copia del DataFrame 'mexico'
mexico1 = mexico.copy()

In [137]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=mexico1['price'].max()
Min=mexico1['price'].min()
Limites= [Min, Max]
Limites

[220.0, 8243.0]

In [138]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(105.0, 13283.0, 3)
intervalos

array([  105.,  6694., 13283.])

In [139]:
#Creamos las categorías
categorias= ["cheap", "expensive"]

In [140]:
#Finalmente creamos las categorías en la columna numérica
mexico1['price']=pd.cut(x= mexico1['price'], bins=intervalos, labels= categorias )

In [141]:
#Rellenamos valores nulos
mexico1= mexico1.fillna(method='bfill')
mexicof= mexico1.fillna(method='ffill')

<ipython-input-141-bb2774a8cb25>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexico1= mexico1.fillna(method='bfill')
<ipython-input-141-bb2774a8cb25>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexicof= mexico1.fillna(method='ffill')


In [142]:
mexico1['price'].isnull().sum()

0

In [143]:
vars_Indep = mexico1[['calculated_host_listings_count']]
#'bathrooms'
vars_Dep = mexico1['price']
#'accommodates', 'host_response_rate', 'host_acceptance_rate', 'host_total_listings_count', 'reviews_per_month'

In [144]:
#Redefinimos las variables
x= vars_Indep
y= vars_Dep

In [145]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test, X_train, Y_test, Y_train= train_test_split(x,y, test_size=0.3, random_state=None)

In [146]:
escalar = StandardScaler()

In [147]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [148]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [149]:
#Entrenamos el modelo
algoritmo.fit(X_train, Y_train)

LogisticRegression()

In [150]:
#realizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array(['cheap', 'cheap', 'cheap', ..., 'cheap', 'cheap', 'cheap'],
      dtype=object)

In [151]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test, y_pred)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[18141     0]
 [   69     0]]


In [152]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score

precision = precision_score(Y_test, y_pred, average="binary", pos_label="cheap")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9962108731466227


In [153]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(Y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9962108731466227


In [154]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(Y_test, y_pred, average="binary", pos_label="cheap")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


**2 Categoricas**

In [155]:
# Crear una copia del DataFrame 'mexico'
mexico2 = mexico.copy()

In [156]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=mexico2['accommodates'].max()
Min=mexico2['accommodates'].min()
Limites= [Min, Max]
Limites

[2, 8]

In [157]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos1=np.linspace(0.99, 10.1, 3)
intervalos1

array([ 0.99 ,  5.545, 10.1  ])

In [158]:
#Creamos las categorías
categorias1= ["pocos", "manada"]

In [159]:
mexico2['accommodates']

,accommodates
0,4
1,4
2,2
3,4
4,4
...,...
26010,6
26011,4
26012,2
26013,2


In [160]:

# Obtenemos el límite superior y el límite inferior de la columna objetivo
Max = mexico2['accommodates'].max()
Min = mexico2['accommodates'].min()
Limites = [Min, Max]
print("Límites:", Limites)

# Categorización de variables
# Declaramos 2 intervalos (ajustando los límites para que se adapten a los datos)
intervalos1 = np.linspace(Min, Max, 3)  # Esto creará 2 intervalos
print("Intervalos:", intervalos1)

# Creamos las categorías
categorias1 = ["pocos", "manada"]  # Asegúrate de que la cantidad de categorías sea una menos que la cantidad de intervalos

# Creamos las categorías en la columna 'accommodates'
mexico2['categoría'] = pd.cut(x=mexico2['accommodates'], bins=intervalos1, labels=categorias1, include_lowest=True)

# Visualizamos el resultado
print(mexico2)


Límites: [2, 8]
Intervalos: [2. 5. 8.]
                        id                                          name  \
0                    35797                                   Villa Dante   
1                   898633  City SkyLine View from 1 Bedroom Apt in CDMX   
2                   913084            15 blocks from Paseo de la Reforma   
3                    67703             2 bedroom apt. deco bldg, Condesa   
4                   939721  Radiant Apartment with Terrace in Roma Norte   
...                    ...                                           ...   
26010  1187191423076770832   ¡Crystal! - 7BR Fantastic! In heart of Roma   
26011  1187248376951514656               Iconic Suite | Terrace | View !   
26012  1187269835146549014         Acogedor y moderno depa en la Condesa   
26013  1187283698786191137                    Estudio con amplia terraza   
26014  1187618795284149417       Apt with Reforma View, Pool and Jacuzzi   

         host_id host_name          neighbourhoo

In [161]:
#Finalmente creamos las categorías en la columna numérica
mexico2['accommodates']=pd.cut(x= mexico2['accommodates'], bins=intervalos1, labels= categorias1 )

In [162]:
#Rellenamos valores nulos
mexico2= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')

<ipython-input-162-10e5fe42bce8>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexico2= mexico.fillna(method='bfill')
<ipython-input-162-10e5fe42bce8>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexicof= mexico.fillna(method='ffill')


In [163]:
mexico2['accommodates'].isnull().sum()

0

In [126]:
print(mexico.columns)


Index(['accommodates', 'categoría'], dtype='object')


In [164]:
vars_Indep_1 = mexico2[['calculated_host_listings_count']]
#'bathrooms'
vars_Dep_1 = mexico2['accommodates']
#'accommodates', 'host_response_rate', 'host_acceptance_rate', 'host_total_listings_count', 'reviews_per_month'

In [165]:
#Redefinimos las variables
x= vars_Indep_1
y= vars_Dep_1

In [166]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test1, X_train1, Y_test1, Y_train1= train_test_split(x,y, test_size=0.3, random_state=None)

In [167]:
escalar1 = StandardScaler()

In [168]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train1 = escalar.fit_transform(X_train1)
X_test1 = escalar.transform(X_test1)

In [169]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo1 = LogisticRegression()

In [173]:
#Entrenamos el modelo
algoritmo1.fit(X_train1, Y_train1)

LogisticRegression()

In [174]:
#realizamos una prediccion
y_pred1 = algoritmo1.predict(X_test1)
y_pred1

array([2, 2, 2, ..., 2, 2, 2])

In [175]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test1, y_pred1)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[10324   322     0     0]
 [ 6180   943     0     0]
 [  338    49     0     0]
 [   40    14     0     0]]


In [209]:
# Calculamos la precisión del modelo para clasificación multiclase
from sklearn.metrics import precision_score

precision = precision_score(Y_test1, y_pred1, average="macro")
print('Precisión del modelo:')
print(precision)



Precisión del modelo:
0.33040731965892234


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [211]:

# Calculamos la precisión del modelo para clasificación multiclase
from sklearn.metrics import precision_score
precision = precision_score(Y_test2, y_pred2, average="macro")
print('Precisión del modelo:')
print(precision)


Precisión del modelo:
0.016723709222865483


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [177]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(Y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6187259747391544


In [214]:
# Calculamos la sensibilidad del modelo para clasificación multiclase
from sklearn.metrics import recall_score

sensibilidad = recall_score(Y_test1, y_pred1, average="macro")
print('Sensibilidad del modelo:', sensibilidad)


Sensibilidad del modelo: 0.2755354842313595


**3 Categoricas**

In [179]:
# Crear una copia del DataFrame 'mexico'
mexico3 = mexico.copy()

In [181]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=mexico3['minimum_nights'].max()
Min=mexico3['minimum_nights'].min()
Limites= [Min, Max]
Limites

[1, 1125]

In [182]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos2=np.linspace(-0.1, 7.1, 3)
intervalos2


array([-0.1,  3.5,  7.1])

In [183]:
#Creamos las categorías
categorias2= ["tranquilos", "activos"]

In [184]:
mexico3['minimum_nights']

,minimum_nights
0,1
1,3
2,2
3,2
4,3
...,...
26010,2
26011,1
26012,2
26013,1


In [185]:
#Finalmente creamos las categorías en la columna numérica
mexico3['minimum_nights']=pd.cut(x= mexico3['minimum_nights'], bins=intervalos2, labels= categorias2 )


In [186]:
#Rellenamos valores nulos
mexico3= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')


<ipython-input-186-8310c526da4f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexico3= mexico.fillna(method='bfill')
<ipython-input-186-8310c526da4f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexicof= mexico.fillna(method='ffill')


In [187]:
mexico3['minimum_nights'].isnull().sum()

0

In [189]:

# Usamos 'accommodates' en lugar de 'bedrooms'
vars_Indep_2 = mexico3[['accommodates']]

# Variable dependiente sigue siendo 'minimum_nights'
vars_Dep_2 = mexico3['minimum_nights']


In [190]:
#Redefinimos las variables
x= vars_Indep_2
y= vars_Dep_2


In [191]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test2, X_train2, Y_test2, Y_train2= train_test_split(x,y, test_size=0.3, random_state=None)


In [192]:
escalar2 = StandardScaler()

In [193]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train2 = escalar.fit_transform(X_train2)
X_test2 = escalar.transform(X_test2)


In [194]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo2 = LogisticRegression()


In [195]:
#Entrenamos el modelo
algoritmo2.fit(X_train2, Y_train2)


LogisticRegression()

In [196]:
#realizamos una prediccion
y_pred2 = algoritmo2.predict(X_test2)
y_pred2


array([1, 1, 1, ..., 1, 1, 1])

In [202]:
precision = precision_score(Y_test2, y_pred2, average="macro")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [197]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test2, y_pred2)
print('Matriz de Confusion')
print(matriz)


Matriz de Confusion
[[8708    0    0 ...    0    0    0]
 [4933  258    0 ...    0    0    0]
 [1852   90    0 ...    0    0    0]
 ...
 [   2    0    0 ...    0    0    0]
 [   1    0    0 ...    0    0    0]
 [   1    0    0 ...    0    0    0]]


In [204]:
# Calculamos la precisión del modelo para clasificación multiclase
from sklearn.metrics import precision_score
precision = precision_score(Y_test2, y_pred2, average="macro")
print('Precisión del modelo:')
print(precision)


Precisión del modelo:
0.016723709222865483


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [199]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test2, y_pred2)
print('Exactitud del modelo:')
print(exactitud)


Exactitud del modelo:
0.4923668314113125


In [206]:
# Calculamos la sensibilidad del modelo para clasificación multiclase
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test2, y_pred2, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.016149252404309235


**4 categoricas**

In [201]:
# Crear una copia del DataFrame 'mexico'
mexico4 = mexico.copy()

In [ ]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=mexico['bathrooms'].max()
Min=mexico['bathrooms'].min()
Limites= [Min, Max]
Limites


In [ ]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos3=np.linspace(-0.1, 4.6, 3)
intervalos3


In [ ]:
#Creamos las categorías
categorias3= ["extreñidos", "surrones"]


In [ ]:
mexico['bathrooms']

In [ ]:
#Finalmente creamos las categorías en la columna numérica
mexico['bathrooms']=pd.cut(x= mexico['bathrooms'], bins=intervalos3, labels= categorias3 )


In [ ]:
#Rellenamos valores nulos
mexico= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')


In [ ]:
mexico['bathrooms'].isnull().sum()

In [ ]:
vars_Indep_3 = mexico[['bedrooms']]

vars_Dep_3 = mexico['bathrooms']


In [ ]:
#Redefinimos las variables
x= vars_Indep_3
y= vars_Dep_3


In [ ]:

#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test3, X_train3, Y_test3, Y_train3= train_test_split(x,y, test_size=0.3, random_state=None)


In [ ]:
escalar3 = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train3 = escalar.fit_transform(X_train3)
X_test3 = escalar.transform(X_test3)


In [ ]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo3 = LogisticRegression()


In [ ]:
#Entrenamos el modelo
algoritmo3.fit(X_train3, Y_train3)


In [ ]:
#realizamos una prediccion
y_pred3 = algoritmo3.predict(X_test3)
y_pred3


In [ ]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test3, y_pred3)
print('Matriz de Confusion')
print(matriz)


In [ ]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test3, y_pred3, average="binary", pos_label="extreñidos")
print('Precision del modelo:')
print(precision)


In [ ]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test3, y_pred3)
print('Exactitud del modelo:')
print(exactitud)


In [ ]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test3, y_pred3, average="binary", pos_label="extreñidos")
print('Sensibilidad del modelo:')
print(sensibilidad)


**5 Categoria**

In [ ]:
# Crear una copia del DataFrame 'mexico'
mexico5 = mexico.copy()

In [ ]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=mexico['host_total_listings_count'].max()
Min=mexico['host_total_listings_count'].min()
Limites= [Min, Max]
Limites


In [ ]:
mexico['host_total_listings_count']

In [ ]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos4=np.linspace(0.99, 442.1, 3)
intervalos4


In [ ]:
#Creamos las categorías
categorias4= ["pocas", "muchas"]


In [ ]:
mexico['host_total_listings_count']

In [ ]:
#Finalmente creamos las categorías en la columna numérica
mexico['host_total_listings_count']=pd.cut(x= mexico['host_total_listings_count'], bins=intervalos4, labels= categorias4 )


In [ ]:
#Rellenamos valores nulos
mexico= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')


In [ ]:
mexico['host_total_listings_count'].isnull().sum()

In [ ]:
vars_Indep_4 = mexico[['bedrooms']]

vars_Dep_4 = mexico['host_total_listings_count']


In [ ]:
#Redefinimos las variables
x= vars_Indep_4
y= vars_Dep_4


In [ ]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test4, X_train4, Y_test4, Y_train4= train_test_split(x,y, test_size=0.3, random_state=None)


In [ ]:
escalar4 = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train4 = escalar.fit_transform(X_train4)
X_test4 = escalar.transform(X_test4)


In [ ]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo4 = LogisticRegression()


In [ ]:
#Entrenamos el modelo
algoritmo4.fit(X_train4, Y_train4)


In [ ]:
#realizamos una prediccion
y_pred4 = algoritmo4.predict(X_test4)
y_pred4


In [ ]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test4, y_pred4)
print('Matriz de Confusion')
print(matriz)


In [ ]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test4, y_pred4, average="binary", pos_label="pocas")
print('Precision del modelo:')
print(precision)


In [ ]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test4, y_pred4)
print('Exactitud del modelo:')
print(exactitud)


In [ ]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test4, y_pred4, average="binary", pos_label="pocas")
print('Sensibilidad del modelo:')
print(sensibilidad)


**FIN**